In [9]:
import torch

r = torch.randint(0,1,[3,4])
a = torch.zeros([3,4])
b = a.fill_(0.1)

b.scatter_(1, r.data.unsqueeze(1), 0.1)
b

torch.Size([3, 4])


RuntimeError: Index tensor must have the same number of dimensions as self tensor

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import shutil
import time
import argparse
from tqdm import tqdm

from model_archive.ResNet import ResNet, Config


SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    image.clamp_(min = image_min, max = image_max)
    image.add_(-image_min).div_(image_max - image_min + 1e-5)
    return image    

def calculate_topk_accuracy(y_pred, y, k = 5):
    with torch.no_grad():
        batch_size = y.shape[0]
        _, top_pred = y_pred.topk(k, 1)
        top_pred = top_pred.t()
        correct = top_pred.eq(y.view(1, -1).expand_as(top_pred))
        correct_1 = correct[:1].reshape(-1).float().sum(0, keepdim = True)
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim = True)
        acc_1 = correct_1 / batch_size
        acc_k = correct_k / batch_size
    return acc_1, acc_k

def train(model, iterator, optimizer, criterion, device, scheduler=None):
    
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        if scheduler=='yes':
            scheduler.step()
        
        epoch_loss += loss.item()
        epoch_acc_1 += acc_1.item()
        epoch_acc_5 += acc_5.item()
        
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
        
    return epoch_loss, epoch_acc_1, epoch_acc_5

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
            epoch_acc_5 += acc_5.item()
        
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
        
    return epoch_loss, epoch_acc_1, epoch_acc_5

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


/root/anaconda3/envs/test_jh/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# Not yet for transfer learning

# parser = argparse.ArgumentParser()
# parser.add_argument("--model", type=str, default="resnet50")
# parser.add_argument("--model_path", type=str, required=True)
# # model config 는 뭐고, 왜 저장하는거지?
# parser.add_argument("--batch_size", type=int, default=16)
# parser.add_argument("--root_path", type=str, default='/home/jh/Desktop/VSC/CNN_work/archive/CUB_200_2011/')
# args = parser.parse_args()

print()
print('<< Configurations >>')
print(f'[*] Model       - {args.model}')
print(f'[*] (어떻게 훈련된 모델인지 정보)')


# Dataset
pretrained_size = 224
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

# train_transforms = transforms.Compose([
#                         transforms.Resize(pretrained_size),
#                         transforms.RandomRotation(5),
#                         transforms.RandomHorizontalFlip(0.5),
#                         transforms.RandomCrop(pretrained_size, padding = 10),
#                         transforms.ToTensor(),
#                         transforms.Normalize(mean = pretrained_means, 
#                                                 std = pretrained_stds)
#                     ])
test_transforms = transforms.Compose([
                        transforms.Resize(pretrained_size),
                        transforms.CenterCrop(pretrained_size),
                        transforms.ToTensor(),
                        transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                    ])
ROOT = args.root_path
data_dir = os.path.join(ROOT, 'CUB_200_2011')
images_dir = os.path.join(data_dir, 'images')

BATCH_SIZE = args.batch_size
test_dir = os.path.join(data_dir, 'test')
test_data = datasets.ImageFolder(root = test_dir,
                                transform = test_transforms)
test_iterator = data.DataLoader(test_data, 
                                batch_size = BATCH_SIZE)




<< Configurations >>
[*] Model       - resnet50
[*] (어떻게 훈련된 모델인지 정보)


In [31]:
args = argparse.Namespace(
    model = 'resnet50',
    model_path = '/home/jh/Desktop/VSC/CNN_work/saved/resnet50_bs16_lr0.000125_epochs100_pretrained-no.pt',
    batch_size = 16,
    root_path = '/home/jh/Desktop/VSC/CNN_work/archive/CUB_200_2011/'
)

In [32]:
import torchvision

model = torchvision.models.resnet50(pretrained = False)     # model 달라지면, 바꾸기
IN_FEATURES = model.fc.in_features 
OUTPUT_DIM = len(test_data.classes)
model.fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)

model.load_state_dict(torch.load(args.model_path))

<All keys matched successfully>

In [34]:
'''
Inference, 일단 score만 구해봄. 작동 잘하는듯!
'''

print(f'[*] Model - {args.model}')
print(f'[*] Parameters  - {count_parameters(model):,}')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

writer = SummaryWriter()

# Model training.
best_valid_loss = float('inf')
best_valid_epoch = 0

print('[*] Inference Start !', end='\n\n')

test_loss, test_acc_1, test_acc_5 = evaluate(model, test_iterator, criterion, device)
print(f'\tTest Loss: {test_loss:.3f} | Test Acc @1: {test_acc_1*100:6.2f}% | ' \
        f'Test Acc @5: {test_acc_5*100:6.2f}%')

[*] Parameters  - 23,917,832
[*] Inference Start !

	Test Loss: 2.124 | Test Acc @1:  51.92% | Test Acc @5:  78.34%


- 200개 class, 어떤 순서로 배치된거지?
- 각 image 에 대해서 어떤 확률로 inference 한건지 볼 수 있나?
